### Load necessary libraries

In [1]:
import numpy as np
import pandas as pd
import neuralsens.partial_derivatives as ns
from sklearn.model_selection import train_test_split
import torch
torch.manual_seed(1)
%matplotlib qt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Create synthetic dataset to check behavior of functions

In [3]:
samples = 100000
n_columns = 8
sm = np.random.normal(size=(samples,n_columns))
df = pd.DataFrame(sm, columns=['X' + str(x) for x in range(1,n_columns+1)])

### Check behavior of Jacobian function

#### Create output Y as linear function of inputs with some non-linear relationship

In [4]:
df['Y'] = - 0.8 * df.X1 + 0.5 * df.X2 ** 2 - df.X3 * df.X4 + 0.1 * np.random.normal(size=(samples,)) 

#### Train MLP model using the data.frame created

In [5]:
## Create random 80/20 % split
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'Y'].to_numpy(), df['Y'], test_size = 0.2, random_state = 5)

In [6]:
X_train_tch = torch.FloatTensor(X_train).requires_grad_(True).to(device)
X_test_tch = torch.FloatTensor(X_test).requires_grad_(True).to(device)
y_train_tch = torch.FloatTensor(y_train.to_numpy()).to(device)
y_test_tch = torch.FloatTensor(y_test.to_numpy()).to(device)

#### Define MLP model torch class

In [7]:
class MLP(torch.nn.Sequential):
    def __init__(self, input_size:int, output_size:int = 1, hidden_size:list = [10]):
        # Store layers to initiate sequential neural network
        layers           = []
        first = True
        for idx, neurons in enumerate(hidden_size):
            if first:
                layers += [torch.nn.Linear(input_size, neurons)]
                first = False
            else:
                layers += [torch.nn.Linear(hidden_size[idx-1], neurons)]
            layers += [torch.nn.Sigmoid()]
        layers += [torch.nn.Linear(hidden_size[idx-1], output_size)]
        super(MLP, self).__init__(*layers)

In [8]:
model = MLP(input_size=n_columns, output_size=1, hidden_size=[15,15])
model = model.to(device)

#### Model training

In [9]:
# Define error loss and optimizer
criterion = torch.nn.MSELoss()
lr = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

In [10]:
# Check model performance before training
model.eval()
y_pred = model(X_test_tch)
before_train = criterion(y_pred.squeeze().to(device), y_test_tch)
print('Test loss before training' , before_train.item()) 

Test loss before training 2.2593350410461426


In [11]:
# Train model
model.train()
epoch = 0
loss = before_train
path=[]
while loss.item() > 0.05:
    optimizer.zero_grad() # Reset the gradient
    epoch += 1
    # Forward pass
    y_pred = model(X_train_tch)
    # Compute Loss
    loss = criterion(y_pred.squeeze().to(device), y_train_tch)
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 1: train loss: 2.3066117763519287
Epoch 2: train loss: 2.1761205196380615
Epoch 3: train loss: 2.1755361557006836
Epoch 4: train loss: 2.175203561782837
Epoch 5: train loss: 2.1748738288879395
Epoch 6: train loss: 2.174546480178833
Epoch 7: train loss: 2.1742215156555176
Epoch 8: train loss: 2.173898458480835
Epoch 9: train loss: 2.173577308654785
Epoch 10: train loss: 2.173258066177368
Epoch 11: train loss: 2.172940254211426
Epoch 12: train loss: 2.172624111175537
Epoch 13: train loss: 2.172309398651123
Epoch 14: train loss: 2.1719958782196045
Epoch 15: train loss: 2.1716835498809814
Epoch 16: train loss: 2.1713719367980957
Epoch 17: train loss: 2.1710612773895264
Epoch 18: train loss: 2.1707513332366943
Epoch 19: train loss: 2.1704418659210205
Epoch 20: train loss: 2.170133113861084
Epoch 21: train loss: 2.1698243618011475
Epoch 22: train loss: 2.16951584815979
Epoch 23: train loss: 2.1692073345184326
Epoch 24: train loss: 2.168898820877075
Epoch 25: train loss: 2.1685900688171

In [12]:
# Check model performance after training
model.eval()
y_pred = model(X_test_tch)
before_train = criterion(y_pred.squeeze().to(device), y_test_tch)
print('Test loss after training' , before_train.item())  

Test loss after training 0.04676543548703194


#### Execute jacobian function from pytorch

In [30]:

try:
    jac = torch.empty(X_train_tch.shape).to(device)
    for i in np.arange(0,jac.shape[0]):
        jac[i,] = torch.autograd.functional.jacobian(model.to(device), X_train_tch[i,].to(device))
except RuntimeError: # CUDA out of memory
    print("CUDA out of memory, cpu would be used instead")
    dev_jac = torch.device('cpu')
    X_train_jac = X_train_tch.to(dev_jac)
    model_jac = model.to(dev_jac)
    jac = torch.empty(X_train_tch.shape)
    for i in np.arange(0,jac.shape[0]):
        jac[i,] = torch.autograd.functional.jacobian(model_jac, X_train_jac[i,])
    

In [32]:
try:
    ns.alpha_sens_curves(jac.detach().numpy(), max_alpha = 300)
except TypeError: # Jacobian is in cuda
    ns.alpha_sens_curves(jac.cpu().detach().numpy(), max_alpha = 300)


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
